In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from torchvision import transforms, models, datasets
import torchvision
import numpy as np
import h5py
from matplotlib import pyplot as plt
from Utils import save_large_dataset, load_large_dataset, calculate_metric, PAC2018Dataset
import pandas as pd
plt.ion()   # interactive mode

# Loading MRI images and labels

The orginal dataset was loaded with nibabel library and saved into files. See data_loader.ipynb file for details.

In [2]:
X = load_large_dataset('images')
Y = load_large_dataset('labels')

In [3]:
X = X.squeeze() #remove unnecessary dimension
Y = Y.squeeze()
print (X.shape)

(1792, 121, 145, 121)


# Extra covariates

In the final model we use only scanner ID as the extra covariates

In [4]:
df = pd.read_csv('PAC2018_Sites.csv', delimiter=';')
scanner_id = df.values[:,1:2]
extra_features = np.zeros((len(Y),3))

In [5]:
for i in range(len(Y)):
    if (scanner_id[i]==1): extra_features[i][0]=1
    if (scanner_id[i]==2): extra_features[i][1]=1
    if (scanner_id[i]==3): extra_features[i][2]=1

In [6]:
extra_features = extra_features.astype('float32')

# Model

We use 2D-convnet architecture, namely Resnet-18 with pretrained weights. Last layer (fully-connected classifier) was enhanced and 3 extra layers are added along with dropout. According to our experiments, it gives a bit better result than the original Resnet-18.

In [9]:
class Resnet_extra_features(nn.Module):
            def __init__(self):
                super(Resnet_extra_features, self).__init__()
                self.features_from_Resnet = nn.Sequential(*list(resnet_pretrained.children())[:-1])
                
                self.classifier = nn.Sequential(
                nn.Linear(num_ftrs+3, 4096), #input is features of an image plus 3 extra features (scan id)
                nn.ReLU(True),
                nn.Dropout(),
                nn.Linear(4096, 4096),
                nn.ReLU(True),
                nn.Dropout(),
                nn.Linear(4096, 2),
        )
                
            def forward(self, image, extra_features):
                x = self.features_from_Resnet(image)
                x = x.view(x.size(0), -1) #flatten
                concatenated = torch.cat((x, extra_features), 1)
                z = self.classifier(concatenated)
                return z


The original 3d images were sliced into 2d images, each one with 3 channels ("slices"). We consider 2d images in all 3 dimensions. Regions, which obtained accuracy higher than 61% were selected to the final ensemble model. Here is the list: 

In [7]:
list_classifiers = ["1_22_25", "1_43_46", "1_49_52", "1_52_55", "1_58_61", "1_70_73", "1_85_88", "1_88_91", "1_97_100", "1_100_103",
                    "2_25_28", "2_34_37", "2_55_58", "2_73_76", "2_94_97", "2_97_100", 
                    "3_10_13", "3_13_16", "3_19_22", "3_22_25", "3_25_28", "3_34_37", "3_40_43"]

In [8]:
CUDA = True
BATCH_SIZE = 64

We trained and validated results on 10 randomly selected validation sets and report the average.

In [10]:
for r in range(10):

    np.random.seed(r) 
    mask = np.random.rand(len(X)) < 0.9  #array of boolean variables
    validation_labels = Y[~mask]
    aggregated_preds = np.zeros(len(validation_labels))

    for i in range(len(list_classifiers)):

        axis = int(list_classifiers[i].split("_")[0])
        start_index = int(list_classifiers[i].split("_")[1])
        end_index = int(list_classifiers[i].split("_")[2])

        if (axis==1):
            X_sliced = X[:,start_index:end_index,:,:] #take only 3 slices and treat them as channels
            X_padded = np.pad(X_sliced,((0,0),(0,0),(40,39),(51,52)), 'constant') #pad with zeros to get 224x224
            X_padded = np.float32(X_padded)
            X_padded = np.rollaxis(X_padded, axis, 1) #move channel dimension to be the first one

        if (axis==2):
            X_sliced = X[:,:,start_index:end_index,:] #take only 3 slices and treat them as channels
            X_padded = np.pad(X_sliced,((0,0),(51,52),(0,0),(51,52)), 'constant') #pad with zeros to get 224x224
            X_padded = np.float32(X_padded)
            X_padded = np.rollaxis(X_padded, axis, 1) #move channel dimension to be the first one

        if (axis==3):
            X_sliced = X[:,:,:,start_index:end_index] #take only 3 slices and treat them as channels
            X_padded = np.pad(X_sliced,((0,0),(51,52),(40,39),(0,0)), 'constant') #pad with zeros to get 224x224
            X_padded = np.float32(X_padded)
            X_padded = np.rollaxis(X_padded, axis, 1) #move channel dimension to be the first one


        training_images = X_padded[mask]
        training_extra_features = extra_features[mask]
        training_labels = Y[mask]

        validation_images = X_padded[~mask]
        validation_extra_features = extra_features[~mask]
        validation_labels = Y[~mask]

        training_images = torch.from_numpy(training_images) #convert to torch tensor
        training_extra_features = torch.from_numpy(training_extra_features) #convert to torch tensor
        training_labels = torch.from_numpy(training_labels) #convert to torch tensor

        validation_images = torch.from_numpy(validation_images) #convert to torch tensor
        validation_extra_features = torch.from_numpy(validation_extra_features) #convert to torch tensor
        validation_labels = torch.from_numpy(validation_labels) #convert to torch tensor

        training_labels = training_labels.long()
        validation_labels = validation_labels.long()

        dataset = PAC2018Dataset(training_images, training_extra_features, training_labels)
        train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

        val_set = PAC2018Dataset(validation_images, validation_extra_features, validation_labels)
        validation_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)

        resnet_pretrained =  models.resnet18(pretrained=True)
        num_ftrs = resnet_pretrained.fc.in_features
        net = Resnet_extra_features()

        criterion = nn.CrossEntropyLoss()
        net.cuda()
        optimizer = optim.Adam(net.parameters(), lr=1e-5, weight_decay=1e-3)

        # TRAINING #

        for epoch in range(4):  
            net.train()
            for j, data in enumerate(train_loader, 0):
                    # get the inputs
                    inputs, extra_feat, labels = data

                    # wrap them in Variable
                    if (CUDA):
                        inputs, extra_feat, labels = Variable(inputs.cuda()), Variable(extra_feat.cuda()), Variable(labels.cuda())
                    else:
                        inputs, extra_feat, labels = Variable(inputs), Variable(extra_feat), Variable(labels)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward + backward + optimize
                    outputs = net(inputs,extra_feat)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()


        # EVALUATION #
        net.eval()
        preds = np.zeros((len(validation_labels)))
        for z in range(len(validation_labels)):
            output = net(Variable(val_set[z][0].unsqueeze(0).cuda()), Variable(val_set[z][1].unsqueeze(0).cuda()))
            _, preds[z] = torch.max(output.data.cpu(), 1)
            aggregated_preds[z] += preds[z]

        #print ('Classifier ' + list_classifiers[i] + ': {:.4f}'.format(calculate_metric(preds,validation_labels)))

    ensemble_preds = (aggregated_preds/len(list_classifiers))
    ensemble_preds = np.rint(ensemble_preds) #round to closest integer (0 or 1)
    #print ("----------")
    print ('Metric of Ensemble Model {:.4f}'.format(calculate_metric(ensemble_preds,validation_labels)))
    accuracy = (np.count_nonzero(ensemble_preds==validation_labels))/len(validation_labels)
    #print ('Accuracy of Ensemble Model: {:.4f}'.format(accuracy))

Metric of Ensemble Model 0.6497
Metric of Ensemble Model 0.6245
Metric of Ensemble Model 0.6196
Metric of Ensemble Model 0.6363
Metric of Ensemble Model 0.6788
Metric of Ensemble Model 0.6518
Metric of Ensemble Model 0.6236
Metric of Ensemble Model 0.6276
Metric of Ensemble Model 0.5645
Metric of Ensemble Model 0.6250


The reported metric is (sensitivity+specificity)/2. The average of our results is <b>0.63</b>. On the test set provided by the Photon group we reach <b>0.61</b>

# Things that didn't help

* we tried normal data augmentation techniques (changing brightness, contrast, small distortions)
* we used variational autoencoder to generate more images to augment the original dataset. We extend the size of dataset 5 times, yet did not observe any significant benefits.
* we tried to build the latent represantion with variation autoencoder and then build classifier upon it. We didn't get anything better than 60% accuracy.
* Instead of Resnet, we also experimented with VGG and obtained very similar results
* We tried to incorporate other covariates (gender, age, brain volume) but it didn't help